# Preprocessing
By Jorge Fuentes

The Allen Brain Analysis requires photos to be darkened, straightened, and cropped. These functions aim to acheive this programatically.

Currently only works with .jpg images with rgb values of values 0-255.


## Table of contents
1. [File Helpers](#file)
2. [Crop Image](#crop)
3. [Filter Image](#filter)
4. [Main](#main)

In [8]:
import numpy as np
import imageio
import os
from tqdm import tqdm
from scipy import ndimage

## File Helpers <a name="file"></a>
These functions build off the os library to navigate the file structure

In [9]:
def mkdirsafe (newpath):
    """
    Ensures the path exists, creating it if necessary
    :param str newpath: The path to the directory
    """
    if not os.path.exists(newpath): os.makedirs(newpath)

def filterdirectory(path, extension):
    """
    Find unhidden files with the extension in the path's directory
    :param str path: Path to directory
    :param str extension: File extension on the end of filename
    """
    files = [file for file in os.listdir(path) if file.lower().endswith(extension) and file[0] !='.']
    return files


## Crop Image <a name="crop"></a>
testArr = np.array([[[0,255,255],[255,255,255],[0,255,255]],
                    [[255,255,255],[255,255,255],[255,255,255]], 
                    [[255,255,0],[255,255,255],[255,255,0]]], dtype="uint8")

In [10]:
def maskOfColor(color, img):
    red, green, blue = img[:,:,0], img[:,:,1], img[:,:,2]
    mask = (red == color[0]) & (green == color[1]) & (blue == color[2])
    return mask

In [11]:
def cropImage(picArr, bufferRatio):
    """
    It turns all pure white to pure black, then removes any pure black rows, 
        than adds a buffer that is a bufferRatio percentage of the whole picture.
        
    :param numpy array uint8 picArr: picture array
    :param float bufferRatio: percentage of the images width/height to add to each side
    """
    white = [255, 255, 255]
    black = [0, 0, 0]

    #turn white to black
    picArr[maskOfColor(white, picArr)] = black

    #remove all rows and columns of only black
    bMask = maskOfColor(black, picArr)
    bRows = np.all(bMask, axis=1)
    picArr = picArr[~bRows]
    bMask = maskOfColor(black, picArr)
    bCols = np.all(bMask, axis=0)
    picArr = picArr[:, ~bCols]

    #add rows and cols of black
    rowBuffer = np.zeros((int(round(picArr.shape[0]*bufferRatio)), picArr.shape[1], 3), dtype='uint8')
    picArr = np.vstack((rowBuffer, picArr, rowBuffer))

    colBuffer = np.zeros((picArr.shape[0], int(round(picArr.shape[1]*bufferRatio)), 3), dtype='uint8')
    picArr = np.hstack((colBuffer, picArr, colBuffer))
    return picArr

## Filter Image<a name="filter"></a>

In [12]:
def filterImage(img):
    """
    Take a guassian filter of just the green part of the image to reduce noise
    :param numpy array uint8 imagearray: picture array
    """
    #only look at green values
    img = img[:,:,1]
    #the guassian filter is used to reduce noise by blurring/smoothing the image
    img = ndimage.filters.gaussian_filter(img,2)
    #TODO: add option to compress image
    return img

## Main Code<a name="main"></a>
Change variables below to your values

In [15]:
path_to_images = "/Users/jfuentes/Projects/Allen-Brain-Analysis/idealImages/2495"
bufferRatio = 0.01

In [16]:
os.chdir(path_to_images)
directory = filterdirectory(path_to_images,".jpg")
mkdirsafe('preprocessed')
print(directory)

for picPath in tqdm(directory):
    print(picPath)
    newPicPath = "preprocessed/" + picPath 

    picArr = imageio.imread(picPath)
    picArr = cropImage(picArr, bufferRatio)
    picArr = filterImage(picArr)
    imageio.imwrite(newPicPath, picArr)

  0%|          | 0/61 [00:00<?, ?it/s]

['165_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{4}.jpg', '185_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{6}.jpg', '250_out_{Wendy}{012115}{2495-3}{Jan 21 15, 165035}{ImmunoFluorescentHighRes}{4}.jpg', '405_out_{Wendy}{012115}{2495-5}{Jan 21 15, 165333}{ImmunoFluorescentHighRes}{6}.jpg', '374_out_{Wendy}{012115}{2495-5}{Jan 21 15, 165333}{ImmunoFluorescentHighRes}{3}.jpg', '305_out_{Wendy}{012115}{2495-4}{Jan 21 15, 165204}{ImmunoFluorescentHighRes}{11}.jpg', '125_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{1}.jpg', '105_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{3}.jpg', '153_out_{Wendy}{012115}{2495-1}{Jan 21 15, 164730}{ImmunoFluorescentHighRes}{14}.jpg', '230_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{11}.jpg', '285_out_{Wendy}{012115}{2495-3}{Jan 21 15, 165035}{ImmunoFluorescentHighRes}{10}.jpg', '226_out_{Wendy}{012115}{2495-2}{Jan 2

  2%|▏         | 1/61 [00:08<08:17,  8.30s/it]

185_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{6}.jpg


  3%|▎         | 2/61 [00:15<07:46,  7.90s/it]

...

 97%|█████████▋| 59/61 [10:17<00:20, 10.46s/it]

201_out_{Wendy}{012115}{2495-2}{Jan 21 15, 164904}{ImmunoFluorescentHighRes}{1}.jpg


 98%|█████████▊| 60/61 [10:25<00:10, 10.43s/it]

293_out_{Wendy}{012115}{2495-4}{Jan 21 15, 165204}{ImmunoFluorescentHighRes}{2}.jpg


100%|██████████| 61/61 [10:35<00:00, 10.42s/it]
